In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/lib/python3.7/site-packages (0.0)


In [2]:
!pip install joblib

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
selected_features = df.drop(columns=["koi_disposition",
                            "koi_period_err1", 
                            "koi_period_err2",
                            "koi_time0bk_err1",
                            "koi_time0bk_err2",
                            "koi_steff_err1",
                            "koi_steff_err2",
                            "koi_slogg_err1",
                            "koi_slogg_err2",
                            "koi_srad_err1",
                            "koi_srad_err2"],
                            axis=1)

In [6]:
y = df["koi_disposition"]
X = selected_features

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,133.077240,0.150,0.305,-0.150,3.61600,...,204.89,-103.87,10.8,1,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,132.020050,0.291,0.193,-0.291,2.30900,...,2253.61,-677.78,13.8,1,5855,4.578,0.797,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,134.460380,0.970,0.879,-0.942,79.89690,...,195.16,-64.34,254.3,1,6328,4.481,0.963,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,174.662240,0.300,0.145,-0.300,2.63120,...,7.15,-10.12,38.4,1,4768,4.536,0.779,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,172.258529,0.831,0.016,-0.004,2.22739,...,146.52,-93.21,696.5,1,5712,4.359,1.082,292.16705,48.727589,15.263


In [9]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y_train) 

y_train_encoded = encoder.transform(y_train)
y_test_encoded = encoder.transform(y_test)
y_test_encoded

array([2, 0, 2, ..., 1, 1, 1])

In [10]:
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)
y_train_categorical.shape

(5243, 3)

In [11]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(5243, 30)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units = 30, activation='relu', input_dim=30))
model.add(Dense(units = 30, activation='relu'))
model.add(Dense(units = 3, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                930       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 93        
Total params: 1,953
Trainable params: 1,953
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2)

Train on 5243 samples
Epoch 1/100
5243/5243 - 1s - loss: 0.7541 - accuracy: 0.6338
Epoch 2/100
5243/5243 - 0s - loss: 0.4269 - accuracy: 0.7814
Epoch 3/100
5243/5243 - 0s - loss: 0.3878 - accuracy: 0.7913
Epoch 4/100
5243/5243 - 0s - loss: 0.3766 - accuracy: 0.7967
Epoch 5/100
5243/5243 - 0s - loss: 0.3699 - accuracy: 0.8020
Epoch 6/100
5243/5243 - 0s - loss: 0.3645 - accuracy: 0.8095
Epoch 7/100
5243/5243 - 0s - loss: 0.3575 - accuracy: 0.8112
Epoch 8/100
5243/5243 - 0s - loss: 0.3542 - accuracy: 0.8133
Epoch 9/100
5243/5243 - 0s - loss: 0.3490 - accuracy: 0.8192
Epoch 10/100
5243/5243 - 0s - loss: 0.3461 - accuracy: 0.8211
Epoch 11/100
5243/5243 - 0s - loss: 0.3432 - accuracy: 0.8262
Epoch 12/100
5243/5243 - 0s - loss: 0.3407 - accuracy: 0.8232
Epoch 13/100
5243/5243 - 0s - loss: 0.3358 - accuracy: 0.8322
Epoch 14/100
5243/5243 - 0s - loss: 0.3324 - accuracy: 0.8388
Epoch 15/100
5243/5243 - 0s - loss: 0.3304 - accuracy: 0.8409
Epoch 16/100
5243/5243 - 0s - loss: 0.3259 - accuracy: 0.

In [16]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.2721 - accuracy: 0.8850
Loss: 0.27213324425695146, Accuracy: 0.8850114345550537


In [17]:
predictions = model.predict_classes(X_test_scaled)

predictions_df=pd.DataFrame({"Actual": y_test_encoded, "Prediction": predictions })
results=predictions_df.apply(pd.Series.value_counts)
disposition = ['False Positive', 'Confirmed', 'Candidate',] 
results['Disposition']=disposition

results

,Actual,Prediction,Disposition
2,853,868,False Positive
1,484,559,Confirmed
0,411,321,Candidate


In [18]:
model.save('models/neural_network.h5')